In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Download

In [2]:
!pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 78.1 MB/s eta 0:00:00


## Import libraries

In [3]:
import re
import ast
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim
from torch.optim import Adam
from torchtext.transforms import ToTensor
from torch.utils.data import Dataset, DataLoader
from pyvi import ViTokenizer

# pre-processing

In [4]:
def normalize(text):
    t = text.replace('\n', ' ')
    t = t.lower()
    return t

def delete_hashtag(text):
    return re.sub(r'#\w+', '', text)

def delete_link(text):
    return re.sub(r'http\S+', '', text)

def remove_emojis(text):
    emoj = re.compile(r"""[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF\U00002702-\U000027B0\U000024C2-\U0001F251\U0001f926-\U0001f937\U00010000-\U0010ffff\u200d\u23cf\u23e9\u231a\ufe0f\u3030-]+(?<!\n)""", re.UNICODE)
    return re.sub(emoj, '', text)

def encode_number(text):
    t = text.split(' ')
    t = map(lambda x: '<number>' if bool(re.match(r'^[0-9]+(\.[0-9]+)?$', x)) else x, t)
    return ' '.join(t)

def delete_onelen_token(text):
    t = text.split(' ')
    t = filter(lambda x: len(x)>1, t)
    return ' '.join(t)

def preprocessing(text):
    t = normalize(text)
    t = delete_hashtag(t)
    t = delete_link(t)
    t = remove_emojis(t)
    t = ViTokenizer.tokenize(t)
    t = encode_number(t)
    t = delete_onelen_token(t)
    return t

#Model and Training

In [5]:
%cd /content/drive/MyDrive/CS114

/content/drive/.shortcut-targets-by-id/1Y8ECOyKvn31ywCsUN8dtq1YwMiMpEqep/CS114


## Define class dataset

In [6]:
class HashTag_Dataset(Dataset):
  def __init__(self, root, max_length=250):
    super(HashTag_Dataset, self).__init__()
    self.classes = ['#Q&A', '#cv', '#data', '#deep_learning', '#machine_learning', '#math', '#nlp', '#python', '#sharing']
    text, labels = [], []

    df = pd.read_csv(root, encoding='utf-8-sig')
    texts = df['text']
    labels = df["label"]
    self.texts = texts
    self.labels = labels
    self.vocab = self.make_vocab(texts)
    self.max_length = max_length
  def make_vocab(self, texts):
    vocab = dict()
    for text in texts:
      words = text.split()
      for word in words:
        if word not in vocab:
          vocab[word] = 1
        else:
          vocab[word] += 1
    vocab = list(dict(filter(lambda x: x[1]>3, vocab.items())).keys())
    vocab.append('<UNK>')
    vocab.append('<PAD>')
    return vocab
  def encode_text(self, text):
    words = text.split()
    if len(words) > self.max_length:
      words = words[:self.max_length]
    else:
      words += ['<PAD>']*(self.max_length-len(words))
    enc = [self.vocab.index(w) if w in self.vocab else self.vocab.index('<UNK>') for w in words]
    return enc
  def encode_label(self, label):
    enc = ast.literal_eval(label)
    enc = [1 if l in enc else 0 for l in self.classes]
    return enc
  def __len__(self):
    return len(self.labels)

  def len_vocab(self):
    return len(self.vocab)

  def num_classes(self):
    return len(self.classes)

  def __getitem__(self, idx):
    text = self.texts[idx]
    label = self.labels[idx]
    encode = self.encode_text(text)
    label = self.encode_label(label)
    encode = torch.tensor(encode, dtype=torch.long)
    label = torch.tensor(label, dtype=torch.float32)
    return encode, label

## Create the model

In [7]:
class HashtagRecommendation(nn.Module):
  def __init__(self, embedding_dim, hidden_dim, vocab_size, num_labels):
    super(HashtagRecommendation, self).__init__()
    self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
    self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
    self.fc1 = nn.Linear(hidden_dim, 128)
    self.fc2 = nn.Linear(128, 64)
    self.fc3 = nn.Linear(64, num_labels)
    self.relu = nn.ReLU()
    self.sigmoid = nn.Sigmoid()
    self.dropout = nn.Dropout(p=0.2)

  def forward(self, sentence):
    embeds = self.word_embeddings(sentence)
    gru_out, _ = self.gru(embeds)
    # Select the last output of GRU layer
    gru_out = gru_out[:, -1, :]
    x = self.relu(self.fc1(gru_out))
    x = self.dropout(x)
    x = self.relu(self.fc2(x))
    x = self.dropout(x)
    logits = self.fc3(x)
    probs = self.sigmoid(logits)
    return probs

# Training

In [8]:
def accuracy(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        temp += sum(np.logical_and(y_true[i], y_pred[i])) / sum(np.logical_or(y_true[i], y_pred[i]))
    return temp / y_true.shape[0]

In [9]:
train_set = HashTag_Dataset(root = "train_data.csv")
test_set = HashTag_Dataset(root = "test_data.csv")
test_loader = DataLoader(test_set, batch_size=2)
train_loader = DataLoader(train_set, batch_size=4, shuffle=True, num_workers=2, drop_last=True)
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
num_epochs = 250

In [10]:
model = HashtagRecommendation(embedding_dim=100,hidden_dim=256,num_labels=train_set.num_classes(), vocab_size=train_set.len_vocab())
model = model.to(device)
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=0.0001)
num_iters = len(train_loader)

In [12]:
best_acc = 0

for epoch in range(num_epochs):
    model.train()
    progress_bar = tqdm(train_loader, colour='green')

    for iter, (texts, labels) in enumerate(progress_bar):
        texts = texts.to(device)
        labels = labels.to(dtype=torch.float).to(device)

        # forward
        outputs = model(texts)
        loss_value = criterion(outputs, labels)  # Assuming your model has a single output node
        progress_bar.set_description("Epoch {}/{}. Iteration {}/{}. Loss {:.5f}".format(epoch+1, num_epochs, iter+1, num_iters, loss_value.item()))

        # backward
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()

    # evaluate
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for iter, (texts, labels) in enumerate(progress_bar):
            texts = texts.to(device)
            labels = labels.to(dtype=torch.float).to(device)
            outputs = np.array(model(texts).tolist())
            y_pred.extend((outputs >= 0.5).astype(np.float32))
            y_true.extend(labels.cpu().numpy())

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    acc = accuracy(y_true, y_pred)

    print(f"Accuracy: {acc}")

    if acc > best_acc:
        best_acc = acc
        torch.save(model, '/content/drive/MyDrive/CS114/best_model.pth')

Epoch 1/250. Iteration 262/262. Loss 0.30729: 100%|██████████| 262/262 [00:17<00:00, 14.89it/s]


Accuracy: 0.3652989821882948


Epoch 2/250. Iteration 262/262. Loss 0.45523: 100%|██████████| 262/262 [00:16<00:00, 15.44it/s]


Accuracy: 0.4973759541984724


Epoch 3/250. Iteration 262/262. Loss 0.45006: 100%|██████████| 262/262 [00:17<00:00, 15.30it/s]


...

Accuracy: 0.9562977099236639


Epoch 249/250. Iteration 262/262. Loss 0.12386: 100%|██████████| 262/262 [00:17<00:00, 15.02it/s]


Accuracy: 0.9520038167938923


Epoch 250/250. Iteration 262/262. Loss 0.01448: 100%|██████████| 262/262 [00:18<00:00, 14.50it/s]


Accuracy: 0.9598759541984726


## Evaluate on test set

In [16]:
model = torch.load('/content/drive/MyDrive/CS114/best_model.pth')
model.eval()
y_true = []
y_pred = []
progress_bar = tqdm(test_loader, colour='yellow')
with torch.no_grad():
    for iter, (texts, labels) in enumerate(progress_bar):
        texts = texts.to(device)
        labels = labels.to(dtype=torch.float).to(device)
        outputs = np.array(model(texts).tolist())
        y_pred.extend((outputs >= 0.5).astype(np.float32))
        y_true.extend(labels.cpu().numpy())

y_true = np.array(y_true)
y_pred = np.array(y_pred)
acc = accuracy(y_true, y_pred)

print(f"Accuracy: {acc}")

100%|██████████| 117/117 [00:01<00:00, 79.47it/s]

Accuracy: 0.22542735042735054


#Testing

In [17]:
#load model
model = torch.load('/content/drive/MyDrive/CS114/best_model.pth')
print(model)

HashtagRecommendation(
  (word_embeddings): Embedding(2180, 100)
  (lstm): GRU(100, 256, batch_first=True)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=9, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (dropout): Dropout(p=0.2, inplace=False)
)


In [22]:
sentence = input()

giáo_trình và hướng_dẫn_giải bài_tập xác_suất thống_kê của bách_khoa hà_nội mình xin chia_sẻ với các bạn <number> quyển sách mình từng học học_kì thứ <number> tại đại_học bách_khoa hà_nội nhờ <number> quyển sách này mà sau_này khi học về data science và machine learning mình gần như_không phải ôn lại kiến_thức toán link to pdf


In [25]:
# test = preprocessing(sentence)
test = train_set.encode_text(sentence)
test = ToTensor()(test)[None, :]
test = test.to(device)
pred = model(test)[0].tolist()
pred = [train_set.classes[i] for i in range(len(pred)) if pred[i]>0.5]
print("Predict label: ", *pred)

Predict label:  #machine_learning #sharing
